In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

import tensorflow as tf

/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
from IPython.display import Image

### build a simple NN for  MNIST dataset

load the mnist dataset

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [5]:
n_inputs = 28*28  # MNIST
n_hidden1 = 30
n_outputs = 10 # digits 0-9, 10 classes


reset_graph()
# if batch size is 64, then x is 64 x 784
# if batch size is 128, then x is 128 x 784
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="input")   # "None" is for batch size
y = tf.placeholder(tf.int32, shape=(None), name="y")

x_image = tf.reshape(X,[-1,28,28,1])  # reshpae for 2d conv

In [6]:
learning_rate = 0.01

with tf.name_scope("dnn"):
#     conv1 = tf.layers.conv2d(x_image,  4, kernel_size=3, padding='same', name="conv1", use_bias=False)
#     conv2 = tf.layers.conv2d(conv1,  4, kernel_size=3, padding='same', name="conv2", use_bias=False)
    
    conv1 = tf.layers.conv2d(x_image,  4, kernel_size=3, padding='same', name="conv1", use_bias=True)
    conv2 = tf.layers.conv2d(conv1,  4, kernel_size=3, padding='same', name="conv2", use_bias=True)
    
    hidden1 = tf.layers.dense(tf.contrib.layers.flatten(conv2),   n_hidden1, name="hidden1", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)   

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    
t_vars = tf.trainable_variables()

# conv1_weight = [var for var in t_vars if 'conv1' in var.name  and 'weights' in var.name]
# conv1_w_trans = tf.transpose (conv1_weight, [3, 0, 1, 2])
# conv2_weight = [var for var in t_vars if 'conv2' in var.name  and 'weights' in var.name]
# conv2_w_trans = tf.transpose (conv2_weight, [3, 0, 1, 2])
# tf.summary.image('conv1-w', conv1_w_trans)
# tf.summary.image('conv2-w', conv2_w_trans)


conv1_w = [var for var in t_vars if 'conv1' in var.name  and 'weights' in var.name]
conv2_w = [var for var in t_vars if 'conv2' in var.name  and 'weights' in var.name]
tf.summary.image('conv1-w', conv1_w)
tf.summary.image('conv2-w', conv2_w)



summary = tf.summary.merge_all()

In [7]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [8]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [9]:
n_epochs = 8
batch_size = 50

with tf.Session() as sess:
    
    summary_writer = tf.summary.FileWriter('./log-testFilter/', sess.graph)
    
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            training_v = sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
            #print(loss_v)
            
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        #loss_v = loss.eval(feed_dict={X: X_batch, y: y_batch})
        #[conv1_w, loss_v] = sess.run([conv1_weight, loss], feed_dict={X: X_batch, y: y_batch})
        
        #print("conv1 weights:")
        #print(conv1_w[0])
        #print(type(conv1_w[0]))
        #print(conv1_w[0].shape)
        
        #print('loss :')
        #print(loss_v)
        
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
        #save_path = saver.save(sess, "./my_model_final.ckpt")

        if epoch % 2 == 0:
            # write summary
            summary_str = sess.run(summary, feed_dict={X: X_valid, y: y_valid})
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
        
        
        
    save_path = saver.save(sess, "./model_testFilter.ckpt")

0 Batch accuracy: 0.94 Validation accuracy: 0.9368


InvalidArgumentError: Tensor must be 4-D with last dim 1, 3, or 4, not [0]
	 [[Node: conv2-w = ImageSummary[T=DT_FLOAT, bad_color=Tensor<type: uint8 shape: [4] values: 255 0 0...>, max_images=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv2-w/tag, conv1-w/tensor)]]

Caused by op 'conv2-w', defined at:
  File "/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/leiming/anaconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in st

### visualize filters

In [ ]:
t_vars

In [ ]:
conv1_weight = [var for var in t_vars if 'conv1' in var.name  and 'weights' in var.name]
conv1_weight

In [ ]:
conv1_weight

In [ ]:
conv1_biases = [var for var in t_vars if 'conv1' in var.name  and 'biases' in var.name]
conv1_biases